In [171]:
import pandas as pd
import math 
import scipy
from scipy.stats import norm
import numpy as np

####pricing day is Aug 1st
today=pd.datetime(2019,8,1)
r=0.01

file_name_list=['Corn_Option_08_01','Wheat_Option_08_01',
                'Corn_Future_08_01','Wheat_Future_08_01']

df = pd.DataFrame()


for i in range(len(file_name_list)):

    temp = pd.read_csv('./data/'+file_name_list[i]+'.csv')

    df = pd.concat([df, temp])

##Contract Info:https://www.cmegroup.com/trading/agricultural/grain-and-oilseed/corn_contract_specifications.html?marginsTab=INTER
##one contract have 5000 bushels
df['Maintenance']=df['Maintenance']/5000
##price and strike in file is cent, so devided by 100
df['Underlying_price']=df['Underlying_price']/100
df['STRIKE']=df['STRIKE']/100
df['PRICE']=df['PRICE']/100
##copy a 'MONTH' cloum
df['TIME']=df['MONTH']
##change 19-Oct to 19-10-31
df['MONTH']=df['MONTH'].str.replace('Oct','10-31')
df['MONTH']=df['MONTH'].str.replace('Sep','09-30')
df['MONTH']=df['MONTH'].str.replace('Dec','12-31')
df['MONTH']=df['MONTH'].str.replace('Mar','03-31')
df['MONTH']=df['MONTH'].str.replace('May','05-31')
df['MONTH']=df['MONTH'].str.replace('Jul','07-31')

df.reindex()
df.index=np.arange(df.shape[0])

##the maturity day of futures is 15th 
for i in range(0,len(df)): 
    if df['F/P/C'][i]=='F':
        df['MONTH'][i]=df['MONTH'][i].replace('31','15')
        df['Underlying_price'][i]=df['PRICE'][i]

df['MONTH']=pd.to_datetime(df['MONTH'],format='%y-%m-%d')
##the maturity day of options is the last business day 
for i in range(0,len(df)): 
    if df['F/P/C'][i]!='F':
        try:
            rng=pd.date_range(start=today, end=df['MONTH'][i], freq='B')
            df['MONTH'][i]=rng[-1] 
        except ValueError:
            pass
##actual time 
df['t']=(df['MONTH']-today).dt.days/365   
##time for 16 senerios calculation
df['T']=(df['MONTH']-today-pd.Timedelta(days=1)).dt.days/365

##underlying price in 16 senerios 
s_list='up_s1','down_s1','up_s2','down_s2','up_s3','down_s3','up_s_ex','down_s_ex'
for i in range(0,len(s_list)):
    df[s_list[i]]=np.arange(df.shape[0]).astype('float')

for i in range(0,len(df)): 
    df['up_s1'][i]=(df['Underlying_price'][i]+df['Maintenance'][i]*1/3)
    df['down_s1'][i]=(df['Underlying_price'][i]-df['Maintenance'][i]*1/3)
    df['up_s2'][i]=(df['Underlying_price'][i]+df['Maintenance'][i]*2/3)
    df['down_s2'][i]=(df['Underlying_price'][i]-df['Maintenance'][i]*2/3)
    df['up_s3'][i]=(df['Underlying_price'][i]+df['Maintenance'][i]*3/3)
    df['down_s3'][i]=(df['Underlying_price'][i]-df['Maintenance'][i]*3/3)
    df['up_s_ex'][i]=(df['Underlying_price'][i]+df['Maintenance'][i]*3)
    df['down_s_ex'][i]=(df['Underlying_price'][i]-df['Maintenance'][i]*3)
    
print(df)



/Users/kalaki7up/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Users/kalaki7up/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kalaki7up/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kalaki7up/anaconda3/lib/python3.7/site-packages/ipykernel_laun

   F/P/C      MONTH  Maint. Vol. Scan  Maintenance   PRICE Product  STRIKE  \
0      C 2019-09-30              0.28         0.20  0.2200    Corn    3.75   
1      C 2019-09-30              0.28         0.20  0.1900    Corn    3.80   
2      C 2019-09-30              0.28         0.20  0.1600    Corn    3.85   
3      C 2019-09-30              0.28         0.20  0.1400    Corn    3.90   
4      C 2019-09-30              0.28         0.20  0.1200    Corn    3.95   
5      C 2019-09-30              0.28         0.20  0.1000    Corn    4.00   
6      C 2019-09-30              0.28         0.20  0.0800    Corn    4.05   
7      C 2019-09-30              0.28         0.20  0.0700    Corn    4.10   
8      C 2019-09-30              0.28         0.20  0.0600    Corn    4.15   
9      C 2019-09-30              0.28         0.20  0.0500    Corn    4.20   
10     P 2019-09-30              0.28         0.20  0.0500    Corn    3.75   
11     P 2019-09-30              0.28         0.20  0.0600    Co

In [172]:
def Delta(cp,vol,r,s,k,t):
    if cp=='F':
        return 1
    else:
        d1 = (math.log(s/k)+(r+0.5*vol**2)*t)/(math.sqrt(t)*vol)
        call = norm.cdf(d1)
        put = norm.cdf(-d1)
        if cp=='C':
            return call
        else :
            return put

    
def bs_price(cp,vol,r,s,k,t):
    if cp=='F':
        return s
    else:
        d1 = (math.log(s/k)+(r+0.5*vol**2)*t)/(math.sqrt(t)*vol)
        d2 = d1-vol*math.sqrt(t)
        call = math.exp(-r*t)*(s*norm.cdf(d1)-k*norm.cdf(d2))
        put = math.exp(-r*t)*(-s*norm.cdf(-d1)+k*norm.cdf(-d2))
        if cp=='C':
            return call
        else :
            return put

    
from scipy import optimize
from scipy.optimize import newton

def get_vol_netown(cp,target_value, s, k, t, r):
    if cp!='F':
        price = (lambda vol:bs_price(cp,vol,r,s,k,t)-target_value)
        vol = newton(price,0.5,tol=1.48e-08)
       
        return vol
    else:
        return 1


  
imp_vol=np.zeros(len(df))
for i in range(0,len(df)):
    try:
        imp_vol[i]=get_vol_netown(df['F/P/C'][i],df['PRICE'][i],df['Underlying_price'][i],df['STRIKE'][i],df['t'][i],r)
    except RuntimeError:
        imp_vol[i]=np.nan


df.loc[:,'imp_vol']=imp_vol

df=df[~df['imp_vol'].isin([0,np.nan])]
df.reindex()
df.index=np.arange(df.shape[0])

DELTA=np.zeros(len(df))
for j in range(0,len(df)):
    DELTA[j]=Delta(df['F/P/C'][j],df['imp_vol'][j],r,df['Underlying_price'][j],df['STRIKE'][j],df['T'][j])
df.loc[:,'DELTA']=DELTA

df=df[~df['imp_vol'].isin([0])]
df.reindex()
df.index=np.arange(df.shape[0])

df['up_v']=df['imp_vol']*(1+df['Maint. Vol. Scan'])
df['down_v']=df['imp_vol']*(1-df['Maint. Vol. Scan'])



/Users/kalaki7up/anaconda3/lib/python3.7/site-packages/scipy/optimize/zeros.py:195: RuntimeWarning: Tolerance of -113.64915324088763 reached
  warnings.warn(msg, RuntimeWarning)
/Users/kalaki7up/anaconda3/lib/python3.7/site-packages/scipy/optimize/zeros.py:195: RuntimeWarning: Tolerance of -0.0017156051826179523 reached
  warnings.warn(msg, RuntimeWarning)
/Users/kalaki7up/anaconda3/lib/python3.7/site-packages/scipy/optimize/zeros.py:195: RuntimeWarning: Tolerance of -5649.178392444255 reached
  warnings.warn(msg, RuntimeWarning)


In [173]:

change_list_1=['up_v','down_v','up_v','down_v','up_v','down_v','up_v','down_v','up_v','down_v','up_v','down_v','up_v','down_v','imp_vol','imp_vol']
change_list_2=['Underlying_price','Underlying_price','up_s1','up_s1','down_s1','down_s1','up_s2','up_s2','down_s2','down_s2','up_s3','up_s3','down_s3','down_s3','up_s_ex','down_s_ex']



for i in range(0,16):

    risk_price=np.zeros(len(df))
    m=change_list_1[i]
    name='Senerio_%s' % (i+1)
    n=change_list_2[i]
   
    for j in range(0,len(df)):
        if i not in [14,15]:
                ####(cp,vol,r,s,k,t)
            risk_price[j]=(df['PRICE'][j]-bs_price(df['F/P/C'][j],df[m][j],r,df[n][j],df['STRIKE'][j],df['T'][j]))*5000
            df.loc[:,name]=risk_price
        else:
            risk_price[j]=(df['PRICE'][j]-bs_price(df['F/P/C'][j],df[m][j],r,df[n][j],df['STRIKE'][j],df['T'][j]))*0.33*5000
            df.loc[:,name]=risk_price
            


In [174]:
intramonth_charge_corn=125
intramonth_charge_wheat=275
intercommodity_credit_rate=.55

#####Test case input MUST follow:[0]position,[1]Product,[2]F/P/C',[3] TIME,[4]Underlying,[5]Strike 
#####Test case MUST have and only have 4 positions
#####Position 1 and 2 MUST be corn, postion 3 and 4 MUST be wheat

#test_case_2:

Position_1=[2,'Corn','F','19-Sep']
Position_2=[-1,'Corn','C','19-Oct','19-Dec',4.00]
Position_3=[1,'Wheat','C','19-Sep','19-Sep',4.90]
Posotion_4=[-2,'Wheat','F','19-Dec']
'''
#test_case_1:

Position_1=[2,'Corn','F','19-Sep']
Position_2=[-1,'Corn','F','19-Dec']
Position_3=[1,'Wheat','F','19-Sep']
Posotion_4=[-2,'Wheat','F','19-Dec']
'''
positions=[Position_1,Position_2,Position_3,Posotion_4]

scan_risks,intermonth_charges,intercommodity_credit_total=0,0,0
deltas=[]
Senerios=pd.DataFrame()

for Position in positions:
    if Position[2]!='F':
        p=df.loc[(df['Product']==Position[1])&(df['F/P/C']==Position[2]) 
                  & (df['Underlying']==Position[4])& (df['TIME']==Position[3])
                  & (df['STRIKE']==Position[5])]
        Senerio=p.iloc[:,-16:]*Position[0] 
        delta=p['DELTA'].values*Position[0]
           
    elif Position[2]=='F':
        p=df.loc[(df['Product']==Position[1])&(df['F/P/C']==Position[2])
                  & (df['TIME']==Position[3])]
        Senerio=p.iloc[:,-16:]*Position[0] 
        delta=p['DELTA'].values*Position[0]
        
    Senerios=pd.concat([Senerios,Senerio])
    deltas=np.append(deltas,delta)
    
Senerios.reindex()
Senerios.index=np.arange(Senerios.shape[0])
  
corn_risk=max(Senerios[:2].apply(sum))
print('Scan risk of Corn is %s' % round(corn_risk,2))

wheat_risk=max(Senerios[2:].apply(sum))
print('Scan risk of Wheat is %s' % round(wheat_risk,2) )

risks=[corn_risk,wheat_risk]
if deltas[0]*deltas[1]<0:
    intramonth_charge_corn=min(abs(deltas[0]),abs(deltas[1]))*intramonth_charge_corn
else:
    intramonth_charge_corn=0
if deltas[2]*deltas[3]<0:
    intramonth_charge_wheat=min(abs(deltas[2]),abs(deltas[3]))*intramonth_charge_wheat
else:
    intramonth_charge_wheat=0
print('Delta of position_%d is %s \nDelta of position_%d is %s' % (1,round(deltas[0],2),2,round(deltas[1],2)))
print('Delta of position_%d is %s \nDelta of position_%d is %s' % (3,round(deltas[2],2),4,round(deltas[3],2)))
print('Intramonth_charges_corn is %s' % round(intramonth_charge_corn,2) )
print('Intramonth_charges_wheat is %s' % round(intramonth_charge_wheat,2) )


Senerios.loc[4]=Senerios[:2].apply(sum)
Senerios.loc[5]=Senerios[2:4].apply(sum)


if (deltas[0]+deltas[1])*(deltas[2]+deltas[3])<0:
    import re
    for i in range(0,2):
        if i==0:
            m='corn'
        else:
            m='wheat'
        time_risk=(Senerios.iloc[i+4,0]+Senerios.iloc[i+4,1])/2
        max_idx = Senerios.iloc[i+4,:].idxmax()
        max_idx=int(re.search('\d+', max_idx).group())

        if max_idx in [15,16]:
            pair_idx=max_idx-1
        elif max_idx % 2 == 0:
            pair_idx=max_idx-2
        else:
            pair_idx=max_idx
        print('the active senerio of %s is %s and the pair is %s'% (m,max_idx,pair_idx+1))
        pair_risk=Senerios.iloc[i+4,pair_idx]
        vol_risk=(pair_risk+risks[i])/2-risks[i]
        print('time_risk of %s is %s \nvol_risk of %s is %s'% (m,round(time_risk,2),m,round(vol_risk,2)))
        WPR=((pair_risk+risks[i])/2-time_risk)/abs(positions[i*2][0]+positions[i*2+1][0])
        intercommodity_credit=(WPR*intercommodity_credit_rate
                               *min(abs(positions[0][0]+positions[1][0]),abs(positions[2][0]+positions[3][0])))
        print('WPR of %s is %s \nintercommodity_credit of %s is %s'% (m,round(WPR,2),m,round(intercommodity_credit,2)))
        
        intercommodity_credit_total+=intercommodity_credit
    

print('Intercommodity_credit is %s' % round(intercommodity_credit_total,2) )

MARGIN=intramonth_charge_corn+intramonth_charge_wheat+wheat_risk+corn_risk-intercommodity_credit_total

print('MARGIN is %s' % round(MARGIN,2))

            
       




Scan risk of Corn is 1722.01
Scan risk of Wheat is 2312.89
Delta of position_1 is 2.0 
Delta of position_2 is -0.55
Delta of position_3 is 0.36 
Delta of position_4 is -2.0
Intramonth_charges_corn is 69.35
Intramonth_charges_wheat is 99.69
the active senerio of corn is 13 and the pair is 14
time_risk of corn is -4.27 
vol_risk of corn is -163.8
WPR of corn is 1562.48 
intercommodity_credit of corn is 859.36
the active senerio of wheat is 12 and the pair is 11
time_risk of wheat is 2.15 
vol_risk of wheat is -156.99
WPR of wheat is 2153.75 
intercommodity_credit of wheat is 1184.56
Intercommodity_credit is 2043.92
MARGIN is 2160.01
